In [1]:
import os
import dotenv
import nest_asyncio
from pydantic_ai import Agent, RunContext
from openai import AsyncAzureOpenAI
from pydantic_ai.models.openai import OpenAIModel

# Setup

In [2]:
dotenv.load_dotenv()

True

In [3]:
nest_asyncio.apply()

In [4]:
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")

In [5]:
client = AsyncAzureOpenAI(
    azure_endpoint='https://default-afsd.openai.azure.com/',
    api_version='2024-06-01',
    api_key=AZURE_OPENAI_KEY,
)

In [6]:
model = OpenAIModel('default-gpt-4o', openai_client=client)

In [7]:
weather_agent = Agent(
    model,
    result_type=str,
    system_prompt=(
        "Please check the call check_weather if the user is asking the weather"
    ),
)

In [8]:
question_rewrite_agent = Agent(
    model,
    result_type=str,
    system_prompt=(
        "Rewrite user question to normalize the city name in capitalize given the question"
    ),
)

In [9]:
@weather_agent.tool
async def check_weather(ctx: RunContext[None], city: str) -> str:
    """check weather"""
    return "Sunny" if city == 'Hong Kong' else 'Rainy'

In [10]:
@weather_agent.tool
async def rewrite_question(ctx: RunContext[None], question: str) -> str:
    "Rewrite user question to normalize the city name in full name given the question"
    r = await question_rewrite_agent.run(f"Question: {question}")
    return r.data

In [11]:
result = await weather_agent.run(
    'Hi', deps=None)
print(result.data)

/var/folders/dr/w7p3bgrj7357qqlltyz3qfh80000gn/T/ipykernel_56545/887001836.py:1: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  result = await weather_agent.run(


Hello! How can I assist you today?


In [12]:
result = await weather_agent.run(
    'What is the weather in Hong Kong ?', deps=None)
print(result.data)

The weather in Hong Kong is currently sunny.


In [13]:
result = await weather_agent.run(
    'What is the weather in hk ?', deps=None)
print(result.data)

The weather in Hong Kong is sunny.


In [14]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='Please check the call check_weather if the user is asking the weather', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='What is the weather in hk ?', timestamp=datetime.datetime(2025, 3, 4, 17, 49, 36, 636715, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='rewrite_question', args='{"question": "What is the weather in hk ?"}', tool_call_id='call_Zos2vR21y1lQcLcqh3TCRF51', part_kind='tool-call')], model_name='gpt-4o-2024-05-13', timestamp=datetime.datetime(2025, 3, 4, 17, 49, 36, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='rewrite_question', content='Question: What is the weather in Hong Kong?', tool_call_id='call_Zos2vR21y1lQcLcqh3TCRF51', timestamp=datetime.datetime(2025, 3, 4, 17, 49, 38, 29917, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request'),
 ModelResponse(parts=[To